### Sprint1


In [1]:
### Carregando dados e pré-processamento

import kagglehub

# Download latest version
path = kagglehub.dataset_download("alinaderi1/wind-turbine-gearbox-cm-vibration")

print("Path to dataset files:", path)

/home/amanda/Desktop/XP Educacao/ProjetoFinal/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /home/amanda/.cache/kagglehub/datasets/alinaderi1/wind-turbine-gearbox-cm-vibration/versions/1


In [2]:
import os

for root, dirs, files in os.walk(path):
    for name in files:
        print("File:", os.path.join(root, name))

File: /home/amanda/.cache/kagglehub/datasets/alinaderi1/wind-turbine-gearbox-cm-vibration/versions/1/Healthy_data.csv
File: /home/amanda/.cache/kagglehub/datasets/alinaderi1/wind-turbine-gearbox-cm-vibration/versions/1/Vibration Condition Monitoring Benchmarking Datasets.pdf
File: /home/amanda/.cache/kagglehub/datasets/alinaderi1/wind-turbine-gearbox-cm-vibration/versions/1/Damaged_data.csv


In [3]:
import pandas as pd

df_healthy_data = pd.read_csv(os.path.join(path, "Healthy_data.csv"))
df_damaged_data = pd.read_csv(os.path.join(path, "Damaged_data.csv"))

In [4]:
df_healthy_data = df_healthy_data.dropna()
df_damaged_data = df_damaged_data.dropna()

In [5]:
from scipy.signal import butter, filtfilt
import numpy as np

def lowpass_filter(signal, fs, cutoff=2000, order=4):
    """
    Aplica um filtro passa-baixa Butterworth de 4ª ordem.

    Parâmetros:
        signal (array-like): Sinal de entrada (1D)
        fs (float): Frequência de amostragem (Hz)
        cutoff (float): Frequência de corte do filtro (Hz)
        order (int): Ordem do filtro (padrão: 4)

    Retorna:
        signal_filtrado (np.ndarray): Sinal filtrado
    """
    # Normalizar a frequência de corte (Nyquist = fs / 2)
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq

    # Coeficientes do filtro
    b, a = butter(order, normal_cutoff, btype='low', analog=False)

    # Aplicar filtro com zero-phase (evita atraso de fase)
    filtered_signal = filtfilt(b, a, signal)

    return filtered_signal

In [19]:
all_sensors_acquisitions = []

stages = ['H', 'D']

all_sensors_acquisitions = []

for sensor in range(8):
    for index in range(8):
        for stage in stages:
            if stage == 'H':
                vector = np.array(df_healthy_data[f'H{index+1}_AN{sensor+3}'])
                label = 0
            else:
                vector = np.array(df_damaged_data[f'D{index+1}_AN{sensor+3}'])
                label = index+1
            # acquisitions.append([vector, label])
            all_sensors_acquisitions.append([vector, label])

In [24]:
all_sensors_acquisitions

[[array([-0.82384092, -1.40267958, -2.12245306, ...,  1.04447259,
          0.77290345,  0.16173342], shape=(2400000,)),
  0],
 [array([ 2.54546191,  1.56305935,  0.77869355, ..., -7.10235459,
         -7.56430767, -6.99950763], shape=(2400000,)),
  1],
 [array([-0.24553551, -0.73176668, -1.19468259, ...,  0.13747267,
          1.46187249,  1.46170277], shape=(2400000,)),
  0],
 [array([-5.27273939, -3.0617431 , -0.86998869, ...,  2.64773713,
          1.12770668, -0.7104605 ], shape=(2400000,)),
  2],
 [array([0.80879765, 0.68797896, 0.91611653, ..., 0.33546023, 0.2481363 ,
         0.11553872], shape=(2400000,)),
  0],
 [array([-1.95230454, -2.73323974, -3.49925867, ...,  1.96751081,
          2.77710989,  3.46740438], shape=(2400000,)),
  3],
 [array([-0.00440729,  0.50870214,  0.92270876, ...,  2.41574544,
          1.34643427,  0.70402346], shape=(2400000,)),
  0],
 [array([ 3.45241368,  3.7382071 ,  3.64495635, ..., -0.51978228,
         -1.60025868, -3.56963807], shape=(2400000,

In [26]:
import numpy as np
from sklearn.model_selection import train_test_split

# Split signals and labels
signals = [pair[0] for pair in all_sensors_acquisitions]
labels = [pair[1] for pair in all_sensors_acquisitions]

# Convert to numpy arrays (optional but useful for ML models)
X = np.array(signals)
y = np.array(labels)

# Split into labeled and unlabeled (e.g. 20% labeled, 80% unlabeled)
X_labeled, X_unlabeled, y_labeled, y_unlabeled = train_test_split(
    X, y, train_size=0.2, stratify=y, random_state=42
)

# Initial training set
X_train = X_labeled.copy()
y_train = y_labeled.copy()

# Unlabeled pool (labels can be used later for evaluation)
X_pool = X_unlabeled.copy()

: 

##### Self-Training with Random Forest

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Parameters
confidence_threshold = 0.9
max_iter = 10

# Initial labeled data
X_train = X_labeled.copy()
y_train = y_labeled.copy()

# Unlabeled pool
X_pool = X_unlabeled.copy()

for iteration in range(max_iter):
    print(f"\nIteration {iteration + 1}")
    
    # Train the model
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    
    # Predict probabilities on the pool
    probs = model.predict_proba(X_pool)
    preds = np.argmax(probs, axis=1)
    confidences = np.max(probs, axis=1)
    
    # Select high-confidence samples
    mask = confidences >= confidence_threshold
    if not np.any(mask):
        print("No high-confidence predictions found. Stopping.")
        break

    # Add pseudo-labeled data to training set
    X_pseudo = X_pool[mask]
    y_pseudo = preds[mask]
    
    print(f"Adding {len(y_pseudo)} pseudo-labeled samples.")

    # Update training data
    X_train = np.vstack([X_train, X_pseudo])
    y_train = np.concatenate([y_train, y_pseudo])
    
    # Remove used samples from the pool
    X_pool = X_pool[~mask]

# Evaluate on test set
if 'X_test' in locals():
    y_pred = model.predict(X_test)
    print("\nTest accuracy:", accuracy_score(y_test, y_pred))

##### Autoencoder
